In [ ]:
import ray
GB = 1024 * 1024 * 1024
ray.init(ignore_reinit_error=True, object_store_memory=2 * GB, num_cpus=4), #memory=2*GB)

In [ ]:
import numpy as np
from plangym.ray import ParallelEnvironment

from fragile.core.env import DiscreteEnv
from fragile.core.models import RandomDiscrete, RandomNormal
from fragile.core.states import States
from fragile.core.swarm import NoBalance, Swarm
from fragile.core.walkers import Walkers

from fragile.optimize.env import Function, MinimizerWrapper
from fragile.optimize.benchmarks import EggHolder, StyblinskiTang, Sphere, LennardJones, Rastrigin
from fragile.optimize.models import ESModel, CompasJump, BestCompasJump
from fragile.optimize.swarm import FunctionMapper
from fragile.optimize.critic import GaussianRepulssion

In [ ]:
from fragile.ray.swarm import RemoteSwarm, DistributedSwarm, ParamServer
from fragile.ray.demos import DistributedOptimizer
from fragile.ray.env import Function

In [ ]:
def create_swarm():
    n_atoms = 38
    n_dims = 6
    shape = (3 * n_atoms,) 
    #shape = (6,)
    env_callable = lambda : MinimizerWrapper(LennardJones(n_atoms=n_atoms), options={"maxiter":5})
    #env_callable = lambda : MinimizerWrapper(Rastrigin(shape=shape), options={"maxiter":50})
    model_callable = lambda x: ESModel(shape=shape, scale=0.5, recombination=0.7, mutation=0.8)#RandomNormal(x, low=-2, high=2, loc=0, scale=5)
    ray_env = lambda : Function(env_callable, n_workers=2)
    mapper = Swarm(env=ray_env, model=model_callable, n_walkers=50, max_iters=100000000,
                   minimize=True, accumulate_rewards=False, critic=GaussianRepulssion())
    return mapper

In [ ]:
rs = DistributedOptimizer(create_swarm, n_swarms=4,
                          max_iters_ray=10000, log_every=10, n_comp_add=3,
                          n_param_servers=1, minimize=True, init_reward=10,log_reward=False)

In [ ]:
dmap = rs.plot()
dmap

In [ ]:
rs.run_swarm()